<a href="https://colab.research.google.com/github/ErickGross-19/Machine-Learning-Portfolio/blob/main/CAFA_5_Proteins_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CAFA 5 Proteins Functions**

In [ ]:
!pip install biopython
!pip install bioservices
from bioservices import UniProt
import pandas as pd
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import plotly.express as px
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from Bio import SeqIO


drive.mount('/content/drive')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.1 MB/s eta 0:00:00
  Created wheel for bioservices: filename=bioservices-1.11.2-py3-none-any.whl size=223238 sha256=d5a90f0c5436b50bed44797b8670f4dc2ab8ae83c9dd016aa6fd0f53985879

In [ ]:
Train_terms = pd.read_csv('/content/drive/MyDrive/Train/train_terms.tsv', delimiter='\t')
Train_taxonomy = pd.read_csv('/content/drive/MyDrive/Train/train_taxonomy.tsv', delimiter='\t', encoding='cp1252')
Test_superset_taxonomy_list = pd.read_csv('/content/drive/MyDrive/Test (Targets)/testsuperset-taxon-list.tsv', delimiter='\t', encoding='cp1252')
Train_terms

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


In [ ]:
def read_obo_file(filename):
    with open(filename) as f:
        blocks = f.read().split('\n\n')
    terms = []
    for block in blocks:
        if not block.startswith('[Term]'):
            continue
        term = {}
        for line in block.split('\n'):
            if line.startswith('[Term]'):
                continue
            key, value = line.split(': ', 1)
            term[key] = value
        terms.append(term)
    return terms

terms = read_obo_file('/content/drive/MyDrive/Train/go-basic.obo')
GO_basic = pd.DataFrame(terms)
GO_basic


,id,name,namespace,def,synonym,is_a,alt_id,subset,xref,comment,is_obsolete,consider,relationship,replaced_by
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","""mitochondrial inheritance"" EXACT []",GO:0048311 ! mitochondrion distribution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,GO:0007005 ! mitochondrion organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0000003,reproduction,biological_process,"""The production of new individuals that contai...","""reproductive physiological process"" EXACT []",GO:0008150 ! biological_process,GO:0050876,goslim_plant,Wikipedia:Reproduction,NaN,NaN,NaN,NaN,NaN
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,"""OBSOLETE. Assists in the correct assembly of ...","""ribosomal chaperone activity"" EXACT []",NaN,NaN,NaN,NaN,This term was made obsolete because it refers ...,true,GO:0051082,NaN,NaN
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...","""high-affinity zinc uptake transmembrane trans...",GO:0005385 ! zinc ion transmembrane transporte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47412,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways involving...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...",GO:0009225 ! nucleotide-sugar metabolic process,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47413,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...",GO:2001313 ! UDP-4-deoxy-4-formamido-beta-L-ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47414,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...",GO:2001313 ! UDP-4-deoxy-4-formamido-beta-L-ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47415,GO:2001316,kojic acid metabolic process,biological_process,"""The chemical reactions and pathways involving...","""kojic acid metabolism"" EXACT [GOC:obol]",GO:1901360 ! organic cyclic compound metabolic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Train_taxonomy

,EntryID,taxonomyID
0,Q8IXT2,9606
1,Q04418,559292
2,A8DYA3,7227
3,Q9UUI3,284812
4,Q57ZS4,185431
...,...,...
142241,Q5TD07,9606
142242,A8BB17,7955
142243,A0A2R8QBB1,7955
142244,P0CT72,284812


In [ ]:
GO_basic

,id,name,namespace,def,synonym,is_a,alt_id,subset,xref,comment,is_obsolete,consider,relationship,replaced_by
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","""mitochondrial inheritance"" EXACT []",GO:0048311 ! mitochondrion distribution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,GO:0007005 ! mitochondrion organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0000003,reproduction,biological_process,"""The production of new individuals that contai...","""reproductive physiological process"" EXACT []",GO:0008150 ! biological_process,GO:0050876,goslim_plant,Wikipedia:Reproduction,NaN,NaN,NaN,NaN,NaN
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,"""OBSOLETE. Assists in the correct assembly of ...","""ribosomal chaperone activity"" EXACT []",NaN,NaN,NaN,NaN,This term was made obsolete because it refers ...,true,GO:0051082,NaN,NaN
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...","""high-affinity zinc uptake transmembrane trans...",GO:0005385 ! zinc ion transmembrane transporte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47412,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways involving...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...",GO:0009225 ! nucleotide-sugar metabolic process,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47413,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...",GO:2001313 ! UDP-4-deoxy-4-formamido-beta-L-ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47414,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...",GO:2001313 ! UDP-4-deoxy-4-formamido-beta-L-ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47415,GO:2001316,kojic acid metabolic process,biological_process,"""The chemical reactions and pathways involving...","""kojic acid metabolism"" EXACT [GOC:obol]",GO:1901360 ! organic cyclic compound metabolic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Test_superset_taxonomy_list

,ID,Species
0,9606,homo sapiens[All Names]
1,10090,mus musculus[All Names]
2,10116,Rattus norvegicus
3,3702,Arabidopsis thaliana[All Names]
4,83333,Escherichia coli K-12[all names]
...,...,...
85,8671,Pseudechis porphyriacus (snakes)
86,8673,Pseudonaja textilis (snakes)
87,930089,Bipolaris zeicola 26-R-13 (ascomycetes fungi)
88,559292,Saccharomyces cerevisiae S288C (budding yeast)


In [ ]:
fasta_sequences = SeqIO.parse(open('/content/drive/MyDrive/Train/train_sequences.fasta'),'fasta')
sequence_data = []
for fasta in fasta_sequences:
    sequence_data.append({'id': fasta.id, 'sequence': str(fasta.seq)})
Train_sequences = pd.DataFrame(sequence_data)
Train_sequences


,id,sequence
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
1,O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
2,O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
3,A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
4,P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...
...,...,...
142241,A0A286YAI0,METEVDDFPGKASIFSQVNPLYSNNMKLCEAERYDFQHSEPKTMKS...
142242,A0A1D5NUC4,MSAAASAEMIETPPVLNFEEIDYKEIEVEEVVGRGAFGVVCKAKWR...
142243,Q5RGB0,MADKGPILTSVIIFYLSIGAAIFQILEEPNLNSAVDDYKNKTNNLL...
142244,A0A2R8QMZ5,MGRKKIQITRIMDERNRQVTFTKRKFGLMKKAYELSVLCDCEIALI...


In [ ]:
fasta_sequences = SeqIO.parse(open('/content/drive/MyDrive/Test (Targets)/testsuperset.fasta'),'fasta')
sequence_data = []
for fasta in fasta_sequences:
    sequence_data.append({'id': fasta.id, 'sequence': str(fasta.seq)})
Test_superset = pd.DataFrame(sequence_data)
Test_superset

,id,sequence
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...
...,...,...
141860,P08380,GNCKCDDEGPNVRTAPLTGYVDLGYCNEGWEKCASYYSPIAECCRKKK
141861,C0HK72,RGICLEPKVVGPCKARIRRFYYDSETGKCTPFIYGGCGGNGNNFET...
141862,C0HK73,GSICLEPKVVGPCKAGIRRFYFDSETGKCTLFLYGGCKGNGNNFET...
141863,C0HK74,GSICLEPKVVGPCTAYFPRFYFDSETGKCTPFIYGGCEGNGNNFET...


In [ ]:
Train_Sequence_Taxonomy = pd.merge(Train_sequences, Train_taxonomy, left_on='id', right_on='EntryID')
Train_Sequence_Taxonomy.drop('id', axis=1, inplace=True)
#Train_Sequence_Taxonomy['Species'] = Train_Sequence_Taxonomy['taxonomyID'].apply(lambda x: Test_superset_taxonomy_list.loc[Test_superset_taxonomy_list['ID'] == x, 'Species'].values[0] if x in Test_superset_taxonomy_list['ID'].tolist() else '')

Train_Sequence_Taxonomy

,sequence,EntryID,taxonomyID
0,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,P20536,10249
1,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...,O73864,7955
2,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...,O95231,9606
3,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...,A0A0B4J1F4,10090
4,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...,P54366,7227
...,...,...,...
142241,METEVDDFPGKASIFSQVNPLYSNNMKLCEAERYDFQHSEPKTMKS...,A0A286YAI0,7955
142242,MSAAASAEMIETPPVLNFEEIDYKEIEVEEVVGRGAFGVVCKAKWR...,A0A1D5NUC4,9031
142243,MADKGPILTSVIIFYLSIGAAIFQILEEPNLNSAVDDYKNKTNNLL...,Q5RGB0,7955
142244,MGRKKIQITRIMDERNRQVTFTKRKFGLMKKAYELSVLCDCEIALI...,A0A2R8QMZ5,7955


In [ ]:
file_path = 'path/to/your/file.txt'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
first_100000 = Test_superset['id'][:100000]
leftovers = Test_superset['id'][100000:]

# write the values to separate text files
with open('first_100000.txt', 'w') as f:
    f.write('\n'.join(first_100000.astype(str)))

with open('leftovers.txt', 'w') as f:
    f.write('\n'.join(leftovers.astype(str)))

Test_superset_taxa1 = pd.read_csv('/content/drive/MyDrive/uniprot-download_true_fields_accession_2Corganism_id_format_tsv-2023.04.21-16.34.26.19.tsv', delimiter='\t')
Test_superset_taxa2 = pd.read_csv('/content/drive/MyDrive/uniprot-download_true_fields_accession_2Corganism_id_format_tsv-2023.04.21-16.34.40.14 (1).tsv', delimiter='\t')
print(len(Test_superset_taxa1))
print(len(Test_superset_taxa2))
Test_superset_taxa_list = pd.concat([Test_superset_taxa1, Test_superset_taxa2], axis=0, join='outer')
Test_superset_taxa_list.drop('From', axis=1, inplace=True)
print(Test_superset_taxa_list.columns)

Test_superset_taxa = pd.merge(Test_superset, Test_superset_taxa_list, left_on='id', right_on='Entry')
Test_superset_taxa.drop('Entry',axis=1,  inplace=True)
Test_superset_taxa.rename(columns={'Organism (ID)':"taxonomyID", 'id':'EntryID'}, inplace=True)
Test_superset_taxa

100000
41858
Index(['Entry', 'Organism (ID)'], dtype='object')


,EntryID,sequence,taxonomyID
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,10090
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,10090
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,10090
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,10090
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...,10090
...,...,...,...
141854,P08380,GNCKCDDEGPNVRTAPLTGYVDLGYCNEGWEKCASYYSPIAECCRKKK,38281
141855,C0HK72,RGICLEPKVVGPCKARIRRFYYDSETGKCTPFIYGGCGGNGNNFET...,38281
141856,C0HK73,GSICLEPKVVGPCKAGIRRFYFDSETGKCTLFLYGGCKGNGNNFET...,38281
141857,C0HK74,GSICLEPKVVGPCTAYFPRFYFDSETGKCTPFIYGGCEGNGNNFET...,38281


In [ ]:
import io
import requests

# Define the UniProt-GOA REST API URL and query parameters
url = "https://www.ebi.ac.uk/QuickGO/services/annotation/search"
params = {
    "format": "csv",
    "taxonId": Train_Sequence_Taxonomy['taxonomyID'][1],
    "geneProductType": "protein",
    "qualifier": "enables",
    "limit": "1",
    "page": "1",
    "proteinSequence": Train_Sequence_Taxonomy['sequence'][1],
}



# Send a GET request to the UniProt-GOA REST API
response = requests.get(url, params=params)
print(response.text)

# Extract the GO terms from the response
extracted={}
for line in response.text.split("{"):
    for item in line.split(","):
      values = item.split(":", 1)
      if(len(values)>=2):
        extracted[values[0]] = values[1]

print(extracted)
value_to_remove = ['[', '}']
extracted = {key: value for key, value in extracted.items() if value != value_to_remove}

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame([extracted])
def extract_inside_quotes(text):
    start = text.find('"')
    end = text.rfind('"')
    return text[start+1:end]
df =  df.applymap(extract_inside_quotes)
df = df.applymap(lambda x: np.nan if str(x).strip() == '' else x)
df = df.applymap(lambda x: np.nan if 'nul' in str(x).strip() else x)
df = df.dropna(axis=1, how='all')
df
print(Train_Sequence_Taxonomy['taxonomyID'][1])
df

{"numberOfHits":120779,"results":[{"id":"UniProtKB:A0A024B5K5!489036551","geneProductId":"UniProtKB:A0A024B5K5","qualifier":"enables","goId":"GO:0004222","goName":null,"goEvidence":"IBA","goAspect":"molecular_function","evidenceCode":"ECO:0000318","reference":"PMID:21873635","withFrom":[{"connectedXrefs":[{"db":"FB","id":"FBgn0027570"}]},{"connectedXrefs":[{"db":"MGI","id":"MGI:1343461"}]},{"connectedXrefs":[{"db":"PANTHER","id":"PTN000199753"}]},{"connectedXrefs":[{"db":"RGD","id":"1309299"}]},{"connectedXrefs":[{"db":"UniProtKB","id":"P08473"}]},{"connectedXrefs":[{"db":"UniProtKB","id":"P0DPD6"}]},{"connectedXrefs":[{"db":"UniProtKB","id":"P0DPE2"}]},{"connectedXrefs":[{"db":"UniProtKB","id":"P23276"}]},{"connectedXrefs":[{"db":"UniProtKB","id":"P42891"}]}],"taxonId":7955,"taxonName":null,"assignedBy":"GO_Central","extensions":null,"targetSets":null,"symbol":"ECE2","date":"20200918","synonyms":null,"name":null}],"pageInfo":{"resultsPerPage":1,"current":1,"total":120779}}
{'"numberOf

,"""numberOfHits""","""id""","""geneProductId""","""qualifier""","""goId""","""goEvidence""","""goAspect""","""evidenceCode""","""reference""","""db""","""taxonId""","""assignedBy""","""symbol""","""date""","""total"""
0,12077,P42891,UniProtKB:A0A024B5K5,enables,GO:0004222,IBA,molecular_function,ECO:0000318,PMID:21873635,UniProtKB,795,GO_Central,ECE2,20200918,120779}


In [ ]:
import requests
import pandas as pd

def extract_inside_quotes(text):
    start = text.find('"')
    end = text.rfind('"')
    return text[start+1:end]

def get_UniProt_terms(sequence):
  UniProt_Values = pd.DataFrame()
  url = "https://www.ebi.ac.uk/QuickGO/services/annotation/search"
  for index, row in sequence.iterrows():
    print(index)
    print(row['sequence'])
    params = {
      "format": "csv",
      "geneProductType": "protein",
      "taxonId": row['taxonomyID'],
      "qualifier": "enables",
      "limit": "1",
      "proteinSequence": row['sequence'],
    }
    print(params)
    response = requests.get(url, params=params)
    print(response.text)
        # Extract the GO terms from the response
    extracted = {'id':row['EntryID'],'sequence':row['sequence'], }
    for line in response.text.split("{"):
      for item in line.split(","):
        values = item.split(":", 1)
        if(len(values)>=2):
          extracted[values[0]] = values[1]
                
        # Remove unwanted values
    extracted = {key: value for key, value in extracted.items() if value not in ['[']}

      # Create a DataFrame from the extracted data
    UniProt_Values = pd.concat([UniProt_Values, pd.DataFrame.from_records([extracted])], axis=0, join='outer')
  return UniProt_Values


# Apply the function to the DataFrame

UniProt_Values  = get_UniProt_terms(Train_Sequence_Taxonomy)
UniProt_Values =  UniProt_Values.applymap(extract_inside_quotes)
UniProt_Values = UniProt_Values.applymap(lambda x: np.nan if str(x).strip() == '' else x)
UniProt_Values = UniProt_Values.applymap(lambda x: np.nan if 'nul' in str(x).strip() else x)
UniProt_Values = UniProt_Values.dropna(axis=1, how='all')
pd.DataFrame(UniProt_Values)



UniProt_Values
SequenceGo = UniProt_Values.to_csv('Sequence&GOTerms', index=False)

Streaming output truncated to the last 5000 lines.
{"numberOfHits":281076,"results":[{"id":"UniProtKB:A0A060QMS4!186114358","geneProductId":"UniProtKB:A0A060QMS4","qualifier":"enables","goId":"GO:0008761","goName":null,"goEvidence":"IEA","goAspect":"molecular_function","evidenceCode":"ECO:0000256","reference":"GO_REF:0000002","withFrom":[{"connectedXrefs":[{"db":"InterPro","id":"IPR029767"}]}],"taxonId":1313,"taxonName":null,"assignedBy":"InterPro","extensions":null,"targetSets":null,"symbol":"mnaA","date":"20230306","synonyms":null,"name":null}],"pageInfo":{"resultsPerPage":1,"current":1,"total":281076}}
2446
MSAEGSQSLAAPRGRPSHLLVPARTKTALALLYDEGLENAYDVRLKLTKEVLTIQKQDVVCIGGAPPGANHRTVTLRRQPVGGLGLSIKGGAEHGVPVVISKIFKDQAADQTEMLFIGDAVLQVNGINVENATHEEVVHLLRNAGDDVTITVEYLREAPSFLKLPLGSPGPSSDHSSRASSPLFDSGLHLNGHCSHTAPSSPSSPIANEPKYEKRWLDTLSVPLSMARISRYKAGTEKLRSSALEVLALDGASTGVLQFSTAQDCADWLRSISTNISDLTLQHMKMANKCCSPCDQVVHMGWVNERLQGADNSQNFRPKFLALRGSSFYIFGAPPVSTLDWGRAERAYNLCEVLFKVHKFWLSDNYWLQANLYLGLQDFDCE

In [ ]:
Test_superset_GO = get_UniProt_terms(Test_superset_taxa)
Test_superset_GO = Test_superset_GO.applymap(extract_inside_quotes)
Test_superset_GO = Test_superset_GO.applymap(lambda x: np.nan if str(x).strip() == '' else x)
Test_superset_GO = Test_superset_GO.applymap(lambda x: np.nan if 'nul' in str(x).strip() else x)
Test_superset_GO = Test_superset_GO.dropna(axis=1, how='all')
pd.DataFrame(Test_superset_GO)
SequenceGo = Test_superset_GO.to_csv('Test_Sequence_GOTerms', index=False)

In [ ]:
submission = pd.DataFrame({'Entryid': SequenceGo['id'], 'goId': SequenceGo['goId'], 'Probability': 1})
submission.to_csv('CAFA5Submission.csv', index=False)

# **EDA**

# **Modeling**